In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
"""
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
"""        

'\ngpus = tf.config.experimental.list_physical_devices(\'GPU\')\n\nif gpus:\n    # Restrict TensorFlow to only allocate 4GB of memory on the first GPU\n    try:\n        tf.config.experimental.set_virtual_device_configuration(\n            gpus[0],\n            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])\n        logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n    except RuntimeError as e:\n        # Virtual devices must be set before GPUs have been initialized\n        print(e)\n'

In [3]:
dataset_info = pd.read_csv("crop_face_data.csv")
dataset_info

,file_path,gender,age
0,Dataset\Adience_Dataset\Crop_Faces\0.jpg,f,"(25, 32)"
1,Dataset\Adience_Dataset\Crop_Faces\1.jpg,m,"(25, 32)"
2,Dataset\Adience_Dataset\Crop_Faces\2.jpg,f,"(25, 32)"
3,Dataset\Adience_Dataset\Crop_Faces\4.jpg,m,"(25, 32)"
4,Dataset\Adience_Dataset\Crop_Faces\5.jpg,f,"(25, 32)"
...,...,...,...
13541,Dataset\Adience_Dataset\Crop_Faces\17444.jpg,m,"(25, 32)"
13542,Dataset\Adience_Dataset\Crop_Faces\17447.jpg,m,"(25, 32)"
13543,Dataset\Adience_Dataset\Crop_Faces\17448.jpg,f,"(25, 32)"
13544,Dataset\Adience_Dataset\Crop_Faces\17449.jpg,f,"(25, 32)"


In [4]:
data_file_path = dataset_info['file_path'].tolist()
gender = dataset_info['gender'].tolist()
age = dataset_info['age'].tolist()

In [5]:
le_gender = LabelEncoder( ).fit_transform(gender)

gender = tf.keras.utils.to_categorical(le_gender, num_classes=2)
gender

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
dataset_info['age'].value_counts()

(25, 32)     4352
(38, 43)     2240
(8, 13)      1718
(4, 6)       1505
(15, 20)     1471
(0, 2)        893
(48, 53)      714
(60, 100)     653
Name: age, dtype: int64

In [7]:
le_age = LabelEncoder( ).fit_transform(age)

age = tf.keras.utils.to_categorical(le_age, num_classes=8)
age

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [8]:
file_path_train, file_path_val, y_train, y_val = train_test_split(data_file_path, gender, 
                                                                  test_size=0.25, 
                                                                  random_state=777, 
                                                                  stratify = gender)

In [9]:
print( len(file_path_train) , len(y_train) )

10159 10159


In [10]:
print( len(file_path_val) , len(y_val) )

3387 3387


In [11]:
y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [12]:
BATCH_SIZE = 8

In [13]:
def load_image( image_path , label ):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img , label

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices( (file_path_train , y_train) )
val_dataset = tf.data.Dataset.from_tensor_slices( (file_path_val , y_val) )

In [15]:
train_dataset = train_dataset.map( load_image, 
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .cache()\
                .prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.map( load_image, 
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .cache()\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
train_dataset.take(1)

<TakeDataset shapes: ((None, 224, 224, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [17]:
val_dataset.take(1)

<TakeDataset shapes: ((None, 224, 224, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [18]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler

In [19]:
EfficientNet = tf.keras.applications.efficientnet.EfficientNetB0(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [20]:
#EfficientNet.trainable = False

In [21]:
DROP_OUT_RATE = 0.2

In [22]:
model= Sequential()
model.add( EfficientNet )
model.add( GlobalAveragePooling2D() ) 
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(128, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(2, activation='softmax') )

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               5

In [24]:
initial_learning_rate = 0.01

def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.math.exp(-k*epoch)

In [25]:
lr_scheduler = LearningRateScheduler(lr_exp_decay, verbose=1)

In [26]:
log_dir = os.path.join('Logs')
CHECKPOINT_PATH = os.path.join('CheckPoints')
tb_callback = TensorBoard(log_dir=log_dir)

cp = ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_categorical_accuracy',
                     save_best_only = True,
                     verbose = 1)

In [28]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    #loss='categorical_crossentropy',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=['categorical_accuracy'],
)

In [29]:
hist = model.fit(train_dataset,
                 validation_data=val_dataset,
                 callbacks=[lr_scheduler , cp , tb_callback],
                 epochs = 20,
                 verbose = 1 
)

C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/20

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
1270/1270 [==============================] - 377s 288ms/step - loss: 0.7432 - categorical_accuracy: 0.5019 - val_loss: 0.6925 - val_categorical_accuracy: 0.5276

Epoch 00001: val_categorical_accuracy improved from -inf to 0.52761, saving model to CheckPoints


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 2/20

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009048374180359595.
1270/1270 [==============================] - 364s 286ms/step - loss: 0.6943 - categorical_accuracy: 0.5216 - val_loss: 0.6829 - val_categorical_accuracy: 0.5515

Epoch 00002: val_categorical_accuracy improved from 0.52761 to 0.55152, saving model to CheckPoints


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


...

INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 12/20

Epoch 00012: LearningRateScheduler reducing learning rate to 0.003328710836980796.
1270/1270 [==============================] - 358s 282ms/step - loss: 0.4922 - categorical_accuracy: 0.7516 - val_loss: 0.4363 - val_categorical_accuracy: 0.7916

Epoch 00012: val_categorical_accuracy did not improve from 0.82817
Epoch 13/20

Epoch 00013: LearningRateScheduler reducing learning rate to 0.0030119421191220205.
1270/1270 [==============================] - 362s 285ms/step - loss: 0.4395 - categorical_accuracy: 0.7962 - val_loss: 0.3524 - val_categorical_accuracy: 0.8453

Epoch 00013: val_categorical_accuracy improved from 0.82817 to 0.84529, saving model to CheckPoints


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 14/20

Epoch 00014: LearningRateScheduler reducing learning rate to 0.002725317930340126.
1270/1270 [==============================] - 360s 283ms/step - loss: 0.3803 - categorical_accuracy: 0.8349 - val_loss: 0.3196 - val_categorical_accuracy: 0.8612

Epoch 00014: val_categorical_accuracy improved from 0.84529 to 0.86123, saving model to CheckPoints


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 15/20

Epoch 00015: LearningRateScheduler reducing learning rate to 0.0024659696394160645.
1270/1270 [==============================] - 360s 283ms/step - loss: 0.3667 - categorical_accuracy: 0.8394 - val_loss: 0.3456 - val_categorical_accuracy: 0.8435

Epoch 00015: val_categorical_accuracy did not improve from 0.86123
Epoch 16/20

Epoch 00016: LearningRateScheduler reducing learning rate to 0.0022313016014842983.
1270/1270 [==============================] - 364s 286ms/step - loss: 0.3458 - categorical_accuracy: 0.8473 - val_loss: 0.3210 - val_categorical_accuracy: 0.8598

Epoch 00016: val_categorical_accuracy did not improve from 0.86123
Epoch 17/20

Epoch 00017: LearningRateScheduler reducing learning rate to 0.002018965179946554.
1270/1270 [==============================] - 366s 288ms/step - loss: 0.3237 - categorical_accuracy: 0.8642 - val_loss: 0.3279 - val_categorical_accuracy: 0.8512

Epoch 00017: val_categorical_accurac